In [1]:
import ee
import geemap

try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

In [1]:
import os


In [2]:
os.getcwd()

'c:\\Users\\walee\\Desktop\\github\\FMSE\\others'

## Exporting S1 Pre, Post, and Zscore

In [3]:
# NEW




# Define the feature collection for each city.
cities = {
    'Shikarpur': ee.FeatureCollection('projects/earthengine-legacy/assets/projects/sat-io/open-datasets/geoboundaries/CGAZ_ADM2')
                  .filter(ee.Filter.eq('shapeName', 'Shikarpur')),
    'Sylhet': ee.FeatureCollection("projects/sat-io/open-datasets/geoboundaries/CGAZ_ADM2")
              .filter(ee.Filter.eq('shapeName', 'Sylhet')),
    'Ernakulam': ee.FeatureCollection('projects/earthengine-legacy/assets/projects/sat-io/open-datasets/geoboundaries/CGAZ_ADM2')
                 .filter(ee.Filter.eq('shapeName', 'Ernakulam')),
    'Nhamatanda': ee.FeatureCollection('projects/earthengine-legacy/assets/projects/sat-io/open-datasets/geoboundaries/CGAZ_ADM2')
                  .filter(ee.Filter.eq('shapeName', 'Nhamatanda'))
}

# Parameters for each city
city_params = {
    'Shikarpur': {'startDate': ee.Date('2022-02-01'), 'endDate': ee.Date('2022-09-01'), 'predays': 60, 'postdays': 7},
    'Sylhet': {'startDate': ee.Date('2022-01-01'), 'endDate': ee.Date('2022-05-17'), 'predays': 60, 'postdays': 10},
    'Ernakulam': {'startDate': ee.Date('2018-01-01'), 'endDate': ee.Date('2018-08-07'), 'predays': 30, 'postdays': 20},
    'Nhamatanda': {'startDate': ee.Date('2019-01-01'), 'endDate': ee.Date('2019-03-19'), 'predays': 60, 'postdays': 1}
}

def get_s1_col(date, days, aoi):
    filters = [
        ee.Filter.listContains("transmitterReceiverPolarisation", "VV"),
        ee.Filter.listContains("transmitterReceiverPolarisation", "VH"),
        ee.Filter.Or(ee.Filter.equals("instrumentMode", "IW"), ee.Filter.equals("instrumentMode", "SM")),
        ee.Filter.bounds(aoi),
        ee.Filter.eq('resolution_meters', 10),
        ee.Filter.date(date, date.advance(days + 1, 'day'))
    ]
    return ee.ImageCollection('COPERNICUS/S1_GRD').filter(filters)

def calc_zscore(s1_pre, s1_post, direction):
    base_mean = s1_pre.filter(ee.Filter.equals('orbitProperties_pass', direction)).mean()
    anom = s1_post.filter(ee.Filter.equals('orbitProperties_pass', direction)).mean().subtract(base_mean)
    base_sd = s1_pre.filter(ee.Filter.equals('orbitProperties_pass', direction)).reduce(ee.Reducer.stdDev())
    return anom.divide(base_sd).set({'system:time_start': s1_post.get('system:time_start')})

def calculate_zscore(s1_pre, s1_post, aoi):
    asc = ee.Filter.eq("orbitProperties_pass", "ASCENDING")
    des = ee.Filter.eq("orbitProperties_pass", "DESCENDING")
    
    cond_asc = s1_pre.filter(asc).size().gt(0).And(s1_post.filter(asc).size().gt(0))
    cond_des = s1_pre.filter(des).size().gt(0).And(s1_post.filter(des).size().gt(0))

        # Calculate z-scores based on available data.
    if cond_asc.getInfo() and cond_des.getInfo():
        zscore_asc = calc_zscore(s1_pre, s1_post, 'ASCENDING')
        zscore_des = calc_zscore(s1_pre, s1_post, 'DESCENDING')
        return ee.ImageCollection.fromImages([zscore_asc, zscore_des]).mean().clip(aoi).toFloat()
    elif cond_asc.getInfo():
        return calc_zscore(s1_pre, s1_post, 'ASCENDING')
    elif cond_des.getInfo():
        return calc_zscore(s1_pre, s1_post, 'DESCENDING')
    else:
        raise ValueError("No available images for both ascending and descending passes.")
    

def export_image(image, description, region, scale):
    task = ee.batch.Export.image.toDrive(
        image=image,
        description=description,
        folder='FMSE_zscore',
        region=region,
        scale=scale,
        maxPixels=1e13
    )
    task.start()
    #monitor_task(task)

#def monitor_task(task):
#    while task.active():
#        print('Task status:', task.status())
#        time.sleep(10)
#    print('Task finished:', task.status())

# Iterate over each city and export the pre, post, and z-score images.
for city, city_shp in cities.items():
    params = city_params[city]
    aoi = city_shp.geometry()
    
    s1_pre = get_s1_col(params['startDate'].advance(-params['predays'], 'day'), params['predays'], aoi)
    s1_post = get_s1_col(params['endDate'], params['postdays'], aoi)
    
    pre_image = s1_pre.mean().clip(aoi).toFloat().rename(['VV', 'VH', 'angle'])
    post_image = s1_post.mean().clip(aoi).toFloat().rename(['VV', 'VH', 'angle'])
    z_score_image = calculate_zscore(s1_pre, s1_post, aoi)
    
    
    pre_image = pre_image.select(['VV', 'VH']).reduce(ee.Reducer.mean())
    z_score_image = z_score_image.select(['VV', 'VH']).reduce(ee.Reducer.mean())
    post_image = post_image.select(['VV', 'VH']).reduce(ee.Reducer.mean())
    
    # z_score_image and post image both 
    
    export_image(pre_image, f'{city}_pre_image', aoi, 10)
    export_image(post_image, f'{city}_post_image', aoi, 10)
    #export_image(z_score_image, f'{city}_z_score_image', aoi, 10)
    print(f'Exporting {city} images Done')


Exporting Shikarpur images Done
Exporting Sylhet images Done
Exporting Ernakulam images Done
Exporting Nhamatanda images Done


## Exporting all variables

In [3]:
# Parameters for each city
city_params = {
    'Shikarpur': {'startDate': ee.Date('2022-03-01'), 'endDate': ee.Date('2022-08-01'), 'predays': 60, 'postdays': 30},
    'Sylhet': {'startDate': ee.Date('2022-01-01'), 'endDate': ee.Date('2022-05-17'), 'predays': 60, 'postdays': 10},
    'Ernakulam': {'startDate': ee.Date('2018-03-01'), 'endDate': ee.Date('2018-08-07'), 'predays': 60, 'postdays': 20},
    'Nhamatanda': {'startDate': ee.Date('2019-01-01'), 'endDate': ee.Date('2019-03-19'), 'predays': 60, 'postdays': 1}
}


# Define the feature collection for each city.
cities = {
    'Shikarpur': ee.FeatureCollection('projects/earthengine-legacy/assets/projects/sat-io/open-datasets/geoboundaries/CGAZ_ADM2')
                  .filter(ee.Filter.eq('shapeName', 'Shikarpur')),
    'Sylhet': ee.FeatureCollection("projects/sat-io/open-datasets/geoboundaries/CGAZ_ADM2")
              .filter(ee.Filter.eq('shapeName', 'Sylhet')),
    'Ernakulam': ee.FeatureCollection('projects/earthengine-legacy/assets/projects/sat-io/open-datasets/geoboundaries/CGAZ_ADM2')
                 .filter(ee.Filter.eq('shapeName', 'Ernakulam')),
    'Nhamatanda': ee.FeatureCollection('projects/earthengine-legacy/assets/projects/sat-io/open-datasets/geoboundaries/CGAZ_ADM2')
                  .filter(ee.Filter.eq('shapeName', 'Nhamatanda'))
}

# Function to apply scale factors to Landsat images.
def apply_scale_factors(image):
    optical_bands = image.select(['SR_B.']).multiply(0.0000275).add(-0.2)
    thermal_bands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
    return image.addBands(optical_bands, None, True).addBands(thermal_bands, None, True)

# Function to prepare Landsat images.
def prepare_landsat_images(aoi, endDate):
    year = ee.Date(endDate).get('year')

    l8 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')\
            .filterBounds(aoi)\
            .filterDate('2013-01-01', '2021-12-31')

    l9 = ee.ImageCollection('LANDSAT/LC09/C02/T1_L2')\
            .filterBounds(aoi)

    landsat_combined = l8.merge(l9)

    landsat_filtered = landsat_combined.filter(ee.Filter.calendarRange(year, year, 'year'))\
                                       .filter(ee.Filter.lt('CLOUD_COVER', 20))\
                                       .map(apply_scale_factors)\
                                       .median()\
                                       .clip(aoi)
    
    return landsat_filtered

# Function to prepare datasets for susceptibility analysis.
def prepare_datasets_for_susceptibility(aoi, landsat_filtered):
    dem_proj = ee.ImageCollection("projects/sat-io/open-datasets/FABDEM")\
            .filterBounds(aoi)\
            .mosaic()\
            .clip(aoi).setDefaultProjection('EPSG:3857', None, 30).rename('elevation')

    slope_proj = ee.Terrain.slope(dem_proj)
    aspect_proj = ee.Terrain.aspect(dem_proj)
    
    dem = dem_proj.addBands(slope_proj).addBands(aspect_proj).reproject(crs='EPSG:4326', scale=30)

    ndvi = landsat_filtered.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')
    ndwi = landsat_filtered.normalizedDifference(['SR_B3', 'SR_B5']).rename('NDWI')
    ndbi = landsat_filtered.normalizedDifference(['SR_B6', 'SR_B5']).rename('NDBI')

    image_sus = landsat_filtered.select(['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7'])\
                                .addBands([ndvi, ndwi, ndbi, dem])\
                                .clip(aoi)\
                                .setDefaultProjection('EPSG:4326')
    
    return image_sus

# Function to export image to Google Drive.
def export_image(image, description, region, scale):
    task = ee.batch.Export.image.toDrive(
        image=image,
        description=description,
        folder='EarthEngineImages',
        region=region,
        scale=scale,
        maxPixels=1e13
    )
    task.start()
    #monitor_task(task)

# Function to monitor the export task.
def monitor_task(task):
    import time
    while task.active():
        print('Task status:', task.status())
        time.sleep(10)
    print('Task finished:', task.status())

# Iterate over each city and export the layers.
for city, city_shp in cities.items():
    aoi = city_shp.geometry()
    params = city_params[city]  # Ensure this dict is defined as per your city-specific parameters
    
    landsat_filtered = prepare_landsat_images(aoi, params['endDate'].format())
    image_sus = prepare_datasets_for_susceptibility(aoi, landsat_filtered)

    # Export individual bands
    bands = ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'NDVI', 'NDWI', 'NDBI', 'elevation', 'slope', 'aspect']
    for band in bands:
        export_image(image_sus.select(band), f'{city}_{band}', aoi, 30)
